<a href="https://colab.research.google.com/github/AlbertMdz/Redes-Neuronales-Avanzadas/blob/master/ProyectoFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pickle
import cv2

In [3]:
from google.colab import drive
drive.mount('/content/drive')                             

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
with open('/content/drive/My Drive/Colab Notebooks/ProyectoFinal/Personality_test.pickle','rb') as f:
    train2=pickle.load(f)

In [0]:
train2[2]

array([[132, 110, 101, ...,  88,  76,  73],
       [130, 109, 102, ...,  97,  83,  77],
       [132, 111, 104, ...,  97,  85,  79],
       ...,
       [ 62,  63,  67, ..., 237, 238, 244],
       [ 65,  68,  72, ..., 241, 242, 242],
       [ 67,  70,  73, ..., 242, 242, 242]], dtype=uint8)

In [0]:
def fourpersonalities(personalidad):
  traductor= {'Analistas': [1,0,0,0], 'Diplomaticos': [0,1,0,0], 'Centinelas': [0,0,1,0], 'Exploradores': [0,0,0,1]}
  try:
    y=traductor[personalidad]
  except:
    y=[0,0,0,0]
  return y


In [0]:
def sixteenpersonalities(personalidad):
  traductor= {'ESTJ_Ejecutivo': 1, 'INFP-Mediador': 2, 'ENFJ-Protagonista': 3, 'ISTJ-Logista': 4, 'ENFP-Activista': 5, 'ENTJ-Comandante': 6, 'ISFP-Aventurero': 7, 'ESTP-Emprendedor': 8, 'ENTP-Innovador': 9, 'ESFJ-Consul': 10, '': 11}
  try:
    y=[0]*16
    y[traductor[personalidad]]=1
  except:
    y=[0]*16
  return y

In [0]:
def reset_graph():
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()

In [0]:
def Neural_network_model(
    keep_rate = 0.8,
    n_classes=10,
    batch_size=100
    ):
    reset_graph()    
    x=tf.placeholder('float',[None,56,56])
    y=tf.placeholder('float')    
    img = tf.reshape(x, shape=[-1, 56, 56, 1])
    tf.summary.image("imagen",img,max_outputs=30)    
    weights = {'W_conv1':tf.Variable(tf.random_normal([5,5,1,16])),
               'W_conv2':tf.Variable(tf.random_normal([5,5,16,32])),
               'W_conv3':tf.Variable(tf.random_normal([5,5,32,64])),
               'W_fc':tf.Variable(tf.random_normal([7*7*64,1024])),
               'out':tf.Variable(tf.random_normal([1024, n_classes]))}

    biases = {'b_conv1':tf.Variable(tf.random_normal([16])),
               'b_conv2':tf.Variable(tf.random_normal([32])),
              'b_conv3':tf.Variable(tf.random_normal([64])),
               'b_fc':tf.Variable(tf.random_normal([1024])),
               'out':tf.Variable(tf.random_normal([n_classes]))}
          
    l1 = tf.nn.conv2d(img, weights['W_conv1'], strides=[1,1,1,1], padding='SAME')
    l1 = tf.add(l1, biases['b_conv1'])
    l1 = tf.nn.relu(l1)
    l1 = tf.nn.max_pool(l1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
        
    l2 = tf.nn.conv2d(l1, weights['W_conv2'], strides=[1,1,1,1], padding='SAME')
    l2 = tf.add(l2, biases['b_conv2'])
    l2 = tf.nn.relu(l2)
    l2 = tf.nn.max_pool(l2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    
    l3 = tf.nn.conv2d(l2, weights['W_conv3'], strides=[1,1,1,1], padding='SAME')
    l3 = tf.add(l3, biases['b_conv3'])
    l3 = tf.nn.relu(l3)
    l3 = tf.nn.max_pool(l3, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    
    fc = tf.reshape(l3,[-1, 7*7*64])
    fc = tf.nn.relu(tf.matmul(fc, weights['W_fc'])+biases['b_fc'])
    
    output = tf.matmul(fc, weights['out'])+biases['out']

    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=output
                                                                   , labels=y) )
    tf.summary.scalar("cost",cost)
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    correct = tf.equal(tf.argmax(output, 1), tf.argmax(y, 1))

    accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
    tf.summary.scalar("accuracy",accuracy)
    summaryppal=tf.summary.merge_all()
    return dict(
              x=x,
              y=y,
              output=output,
              cost=cost,
              optimizer=optimizer,
              accuracy=accuracy,
              summary=summaryppal
              )

In [0]:
def train_neural_network(DNN,N_batch, n_personalities="four", hm_epochs=10,batch_size=20):
  mydir="/content/drive/My Drive/Colab Notebooks/ProyectoFinal/"
  if n_personalities=="four":
    persntraductor=fourpersonalities
    nameofdataset="Personality_"
  elif n_personalities=="sixteen":
    persntraductor=sixteenpersonalities
    nameofdataset="Personality_v2_"
  with tf.Session() as sess:
      sess.run(tf.global_variables_initializer())
      writer = tf.summary.FileWriter("./LOG")
      tf.summary.FileWriter.add_graph(writer,sess.graph)
      for epoch in range(hm_epochs):
          epoch_loss = 0            
          for NB in range(N_batch):
                i=0
                with open(mydir+nameofdataset+str(NB)+'.pickle','rb') as f:
                    content=pickle.load(f)
                    train_x=[cv2.resize(content[i],(56,56)) for i in range(len(content)) if (i%2)==0]
                    train_y=[persntraductor(content[i]) for i in range(len(content)) if not (i%2)==0]
                while i< len(train_x):
                    start = i
                    end = i+batch_size
                    batch_x = train_x[start:end]
                    batch_y = train_y[start:end]
                    i+=batch_size                 
                    feed_dict={DNN["x"]: batch_x, 
                              DNN["y"]: batch_y}
                    _, c, prediction,y   = sess.run([DNN["optimizer"], DNN["cost"]
                                                 , DNN["output"], DNN["y"]],
                                                feed_dict=feed_dict)
                    epoch_loss += c
                    i+=batch_size

          print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)
          if (epoch%3==0):
            s,accuracy,_,_=sess.run([DNN["summary"],DNN["accuracy"],DNN["output"],DNN["y"]],feed_dict={DNN["x"]:batch_x,DNN["y"]:batch_y})
            writer.add_summary(s,epoch)
            print ("Accuracy in testing=", accuracy)

      accuracy,_,_=sess.run([DNN["accuracy"],DNN["output"],DNN["y"]],feed_dict={DNN["x"]:batch_x,DNN["y"]:batch_y})
      print ("Accuracy  Train=", accuracy)      
      with open(mydir+nameofdataset+"test.pickle",'rb') as f:
          content=pickle.load(f)
          test_x=[cv2.resize(content[i],(56,56)) for i in range(len(content)) if (i%2)==0]
          test_y=[persntraductor(content[i]) for i in range(len(content)) if not (i%2)==0]
      prediction,y   = sess.run([DNN["output"], DNN["y"]], feed_dict={DNN["x"]:test_x, DNN["y"]:test_y})
      correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
      accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
      print('Accuracy Test:',accuracy.eval())
      confusion = tf.math.confusion_matrix(labels=np.argmax(y,axis=1),predictions=np.argmax(prediction,axis=1))
      print(confusion.eval()) 

In [0]:
mydir="/content/drive/My Drive/Colab Notebooks/ProyectoFinal/"
translatordic={}
c=ord
for NB in range(3):
                i=0
                with open(mydir+'Personality_'+str(NB)+'.pickle','rb') as f:
                    content=pickle.load(f)
                    train_x=[cv2.resize(content[i],(56,56)) for i in range(len(content)) if (i%2)==0]
                    train_y=[content[i] for i in range(len(content)) if not (i%2)==0]
                    for personality in train_y:
                      if personality in translatordic.keys():
                        translatordic[personality]+=1
                      else:
                        translatordic[personality]=1
                        

In [0]:
print (translatordic)

{'Exploradores': 602, 'Centinelas': 875, 'Analistas': 661, 'Diplomaticos': 912, '': 1}


In [0]:
DNN=Neural_network_model(n_classes=4,batch_size=100)
Example=train_neural_network(DNN,N_batch=3,n_personalities="four",batch_size=100,hm_epochs=20)

Epoch 0 completed out of 20 loss: 1378195576.0
Accuracy in testing= 0.1764706
Epoch 1 completed out of 20 loss: 655632740.0
Epoch 2 completed out of 20 loss: 437384871.5
Epoch 3 completed out of 20 loss: 391918935.0
Accuracy in testing= 0.47058824
Epoch 4 completed out of 20 loss: 360470382.5
Epoch 5 completed out of 20 loss: 300592704.0
Epoch 6 completed out of 20 loss: 237884118.0
Accuracy in testing= 0.64705884
Epoch 7 completed out of 20 loss: 220347920.75
Epoch 8 completed out of 20 loss: 238420728.5
Epoch 9 completed out of 20 loss: 220981473.25
Accuracy in testing= 0.8235294
Epoch 10 completed out of 20 loss: 241449481.0625
Epoch 11 completed out of 20 loss: 207973117.78125
Epoch 12 completed out of 20 loss: 152825164.3125
Accuracy in testing= 0.8235294
Epoch 13 completed out of 20 loss: 114205298.640625
Epoch 14 completed out of 20 loss: 108951264.5
Epoch 15 completed out of 20 loss: 101581735.82421875
Accuracy in testing= 1.0
Epoch 16 completed out of 20 loss: 101879302.941406

In [0]:
DNN=Neural_network_model(n_classes=16,batch_size=30)
Example=train_neural_network(DNN,N_batch=3,n_personalities="sixteen",batch_size=30,hm_epochs=30)

Epoch 0 completed out of 30 loss: 7065600220.0
Epoch 1 completed out of 30 loss: 2696524340.0
Epoch 2 completed out of 30 loss: 1954902862.0
Epoch 3 completed out of 30 loss: 1609620814.0
Epoch 4 completed out of 30 loss: 1373900762.0
Epoch 5 completed out of 30 loss: 1207362061.0
Epoch 6 completed out of 30 loss: 1049709092.0
Epoch 7 completed out of 30 loss: 915963349.0
Epoch 8 completed out of 30 loss: 855129524.5
Epoch 9 completed out of 30 loss: 844438505.0
Epoch 10 completed out of 30 loss: 681395824.0
Epoch 11 completed out of 30 loss: 607003071.5
Epoch 12 completed out of 30 loss: 525278260.0
Epoch 13 completed out of 30 loss: 476478069.0
Epoch 14 completed out of 30 loss: 402604451.25
Epoch 15 completed out of 30 loss: 392454424.25
Epoch 16 completed out of 30 loss: 352683186.25
Epoch 17 completed out of 30 loss: 326842558.25
Epoch 18 completed out of 30 loss: 289337705.1875
Epoch 19 completed out of 30 loss: 251923611.25
Epoch 20 completed out of 30 loss: 232956963.625
Epoch 